In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import transforms, datasets
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
import numpy as np
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix,
                            roc_auc_score, roc_curve, f1_score, precision_score, 
                            recall_score)
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MedicalImageCNN(nn.Module):
    def __init__(self, num_classes=2, input_channels=3):
        super(MedicalImageCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout2d(0.25)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout2d(0.25)
        
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout3 = nn.Dropout2d(0.25)
        
        self.conv7 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn7 = nn.BatchNorm2d(256)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn8 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.dropout4 = nn.Dropout2d(0.25)
        
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.dropout5 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.dropout6 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, num_classes)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        x = self.dropout2(x)
        
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        x = self.dropout3(x)
        
        x = F.relu(self.bn7(self.conv7(x)))
        x = F.relu(self.bn8(self.conv8(x)))
        x = self.pool4(x)
        x = self.dropout4(x)
        
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.fc1(x))
        x = self.dropout5(x)
        x = F.relu(self.fc2(x))
        x = self.dropout6(x)
        x = self.fc3(x)
        
        return x


def create_model(num_classes=2, input_channels=3):
    model = MedicalImageCNN(num_classes=num_classes, input_channels=input_channels)
    return model

In [ ]:
def get_data_loaders(data_dir, batch_size=32, img_size=224, use_weighted_sampler=False, num_workers=4):
    train_transform = transforms.Compose([
        transforms.Resize((int(img_size * 1.1), int(img_size * 1.1))),
        transforms.RandomCrop(img_size),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomGrayscale(p=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.1)
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    train_dataset = datasets.ImageFolder(
        root=os.path.join(data_dir, 'train'),
        transform=train_transform
    )
    
    val_dataset = datasets.ImageFolder(
        root=os.path.join(data_dir, 'val'),
        transform=val_transform
    )
    
    train_sampler = None
    class_weights = None
    
    if use_weighted_sampler:
        class_counts = {}
        for idx, (path, class_idx) in enumerate(train_dataset.samples):
            class_counts[class_idx] = class_counts.get(class_idx, 0) + 1
        
        total_samples = sum(class_counts.values())
        num_classes = len(class_counts)
        class_weights = [total_samples / (num_classes * count) for count in class_counts.values()]
        
        sample_weights = [class_weights[class_idx] for _, class_idx in train_dataset.samples]
        train_sampler = WeightedRandomSampler(
            weights=sample_weights,
            num_samples=len(sample_weights),
            replacement=True
        )
        print(f"✓ WeightedRandomSampler etkinleştirildi")
        print(f"  Ağırlıklar: {dict(zip(train_dataset.classes, class_weights))}")
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=(train_sampler is None),
        sampler=train_sampler,
        num_workers=num_workers,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )
    
    if not class_weights:
        class_counts = {}
        for _, class_idx in train_dataset.samples:
            class_counts[class_idx] = class_counts.get(class_idx, 0) + 1
        total_samples = sum(class_counts.values())
        num_classes = len(class_counts)
        class_weights = [total_samples / (num_classes * count) for count in class_counts.values()]
    
    return train_loader, val_loader, train_dataset.classes, class_weights

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    progress_bar = tqdm(train_loader, desc='Training')
    for images, labels in progress_bar:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
        progress_bar.set_postfix({'loss': loss.item()})
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = accuracy_score(all_labels, all_preds)
    
    return epoch_loss, epoch_acc


def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc='Validation')
        for images, labels in progress_bar:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            
            probs = torch.softmax(outputs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            
            progress_bar.set_postfix({'loss': loss.item()})
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = accuracy_score(all_labels, all_preds)
    
    all_probs = np.array(all_probs)
    
    if len(np.unique(all_labels)) == 2:
        auc_roc = roc_auc_score(all_labels, all_probs[:, 1])
    else:
        auc_roc = roc_auc_score(all_labels, all_probs, multi_class='ovr', average='weighted')
    
    f1 = f1_score(all_labels, all_preds, average='weighted')
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    
    return epoch_loss, epoch_acc, all_preds, all_labels, all_probs, {
        'auc_roc': auc_roc,
        'f1_score': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
def plot_training_history(history, save_path='training_history.png'):
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    axes[0, 0].plot(epochs, history['train_loss'], label='Eğitim Loss', marker='o')
    axes[0, 0].plot(epochs, history['val_loss'], label='Doğrulama Loss', marker='s')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Eğitim ve Doğrulama Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    axes[0, 1].plot(epochs, history['train_acc'], label='Eğitim Accuracy', marker='o')
    axes[0, 1].plot(epochs, history['val_acc'], label='Doğrulama Accuracy', marker='s')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].set_title('Eğitim ve Doğrulama Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    if 'val_auc_roc' in history:
        axes[1, 0].plot(epochs, history['val_auc_roc'], label='Doğrulama AUC-ROC', 
                       marker='s', color='green')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('AUC-ROC')
        axes[1, 0].set_title('Doğrulama AUC-ROC')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
    
    if 'val_f1_score' in history:
        axes[1, 1].plot(epochs, history['val_f1_score'], label='Validation F1-Score', 
                       marker='s', color='purple')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('F1-Score')
        axes[1, 1].set_title('Validation F1-Score')
        axes[1, 1].legend()
        axes[1, 1].grid(True)
    else:
        axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Eğitim grafiği kaydedildi: {save_path}")


def plot_confusion_matrix(y_true, y_pred, class_names, save_path='confusion_matrix.png'):
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Örnek Sayısı'})
    plt.title('Karışıklık Matrisi (Confusion Matrix)')
    plt.ylabel('Gerçek Etiket (True Label)')
    plt.xlabel('Tahmin Edilen Etiket (Predicted Label)')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Karışıklık matrisi kaydedildi: {save_path}")


def plot_roc_curve(y_true, y_probs, class_names, save_path='roc_curve.png'):
    n_classes = len(class_names)
    
    plt.figure(figsize=(10, 8))
    
    if n_classes == 2:
        fpr, tpr, _ = roc_curve(y_true, y_probs[:, 1])
        auc_score = roc_auc_score(y_true, y_probs[:, 1])
        plt.plot(fpr, tpr, label=f'{class_names[1]} (AUC = {auc_score:.3f})', linewidth=2)
    else:
        for i in range(n_classes):
            y_true_binary = (y_true == i).astype(int)
            fpr, tpr, _ = roc_curve(y_true_binary, y_probs[:, i])
            auc_score = roc_auc_score(y_true_binary, y_probs[:, i])
            plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {auc_score:.3f})', linewidth=2)
    
    plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=1)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.title('ROC Eğrisi (ROC Curve)')
    plt.legend(loc='lower right')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"ROC eğrisi kaydedildi: {save_path}")

In [ ]:
CHECKPOINT_PATH = 'checkpoints/best_model.pth'
DATA_DIR = 'data'
BATCH_SIZE = 32
IMG_SIZE = 224

In [ ]:
print(f"\nModel yükleniyor: {CHECKPOINT_PATH}")
checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)

class_names = checkpoint.get('class_names', ['Normal', 'Disease'])
num_classes = len(class_names)

print(f"\nModel Bilgileri:")
print(f"  Model: CNN (MedicalImageCNN)")
print(f"  Sınıf Sayısı: {num_classes}")
print(f"  Sınıflar: {class_names}")
print(f"  En iyi Accuracy: {checkpoint.get('val_acc', 0):.4f}")
print(f"  En iyi AUC-ROC: {checkpoint.get('val_auc_roc', 0):.4f}")
print(f"  Epoch: {checkpoint.get('epoch', 'N/A')}")

model = create_model(
    num_classes=num_classes,
    input_channels=3
)

model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval()

print("\n✓ Model başarıyla yüklendi!")


In [ ]:
print(f"\nModel yükleniyor: {CHECKPOINT_PATH}")
checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)

class_names = checkpoint.get('class_names', ['Normal', 'Disease'])
num_classes = len(class_names)
model_type = checkpoint.get('model_type', 'custom')
model_name = checkpoint.get('model_name', 'custom')

print(f"\nModel Bilgileri:")
print(f"  Model Tipi: {model_type}")
print(f"  Model Adı: {model_name}")
print(f"  Sınıf Sayısı: {num_classes}")
print(f"  Sınıflar: {class_names}")
print(f"  En iyi Accuracy: {checkpoint.get('val_acc', 0):.4f}")
print(f"  En iyi AUC-ROC: {checkpoint.get('val_auc_roc', 0):.4f}")
print(f"  Epoch: {checkpoint.get('epoch', 'N/A')}")

if model_type == 'transfer':
    model = create_model(
        num_classes=num_classes,
        input_channels=3,
        pretrained=False,
        model_type='transfer',
        model_name=model_name,
        freeze_backbone=False
    )
else:
    model = create_model(
        num_classes=num_classes,
        input_channels=3,
        pretrained=False,
        model_type='custom'
    )

model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval()

print("\n✓ Model başarıyla yüklendi!")

In [ ]:
test_path = os.path.join(DATA_DIR, 'test')

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])

if os.path.exists(test_path):
    test_dataset = datasets.ImageFolder(root=test_path, transform=test_transform)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False,
        num_workers=4
    )
    
    print(f"✓ Test verileri yüklendi: {len(test_dataset)} görüntü")
    print(f"  Sınıflar: {test_dataset.classes}")
    
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    print("\nTest seti üzerinde değerlendirme yapılıyor...")
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc='Testing'):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    test_acc = accuracy_score(all_labels, all_preds)
    test_f1 = f1_score(all_labels, all_preds, average='weighted')
    test_precision = precision_score(all_labels, all_preds, average='weighted')
    test_recall = recall_score(all_labels, all_preds, average='weighted')
    
    all_probs = np.array(all_probs)
    if len(np.unique(all_labels)) == 2:
        test_auc = roc_auc_score(all_labels, all_probs[:, 1])
    else:
        test_auc = roc_auc_score(all_labels, all_probs, multi_class='ovr', average='weighted')
    
    print("\n" + "="*70)
    print("Test Seti Sonuçları:")
    print("="*70)
    print(f"Accuracy: {test_acc:.4f}")
    print(f"AUC-ROC: {test_auc:.4f}")
    print(f"F1-Score: {test_f1:.4f}")
    print(f"Precision: {test_precision:.4f}")
    print(f"Recall: {test_recall:.4f}")
    print("="*70)
    
    print("\nSınıflandırma Raporu:")
    print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))
    
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=test_dataset.classes, 
                yticklabels=test_dataset.classes)
    plt.title('Test Seti - Karışıklık Matrisi', fontsize=14, fontweight='bold')
    plt.ylabel('Gerçek Etiket', fontsize=12)
    plt.xlabel('Tahmin Edilen Etiket', fontsize=12)
    plt.tight_layout()
    plt.show()
    
else:
    print("⚠️ Test klasörü bulunamadı.")

In [ ]:
def predict_single_image(image_path, model, class_names, device, img_size=224):
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = F.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probabilities, 1)
    
    predicted_class = class_names[predicted.item()]
    confidence_score = confidence.item()
    
    all_probs = probabilities[0].cpu().numpy()
    prob_dict = {class_names[i]: float(all_probs[i]) for i in range(len(class_names))}
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    axes[0].imshow(image)
    axes[0].set_title(f'Görüntü: {os.path.basename(image_path)}', fontsize=12)
    axes[0].axis('off')
    
    sorted_probs = sorted(prob_dict.items(), key=lambda x: x[1], reverse=True)
    classes = [item[0] for item in sorted_probs]
    probs = [item[1] for item in sorted_probs]
    
    axes[1].barh(classes, probs, color='steelblue')
    axes[1].set_xlabel('Olasılık', fontsize=12)
    axes[1].set_title(f'Tahmin: {predicted_class} ({confidence_score*100:.2f}%)', 
                     fontsize=14, fontweight='bold')
    axes[1].set_xlim([0, 1])
    
    for i, prob in enumerate(probs):
        axes[1].text(prob + 0.01, i, f'{prob:.3f}', va='center')
    
    plt.tight_layout()
    plt.show()
    
    return predicted_class, confidence_score, prob_dict

print("✓ Tahmin fonksiyonu hazır!")

In [ ]:
test_dir = 'test'
if os.path.exists(test_dir):
    print("Test görüntüleri üzerinde tahmin yapılıyor...\n")
    
    for class_name in os.listdir(test_dir):
        class_path = os.path.join(test_dir, class_name)
        if os.path.isdir(class_path):
            images = [f for f in os.listdir(class_path) 
                     if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            if images:
                for img_name in images[:2]:
                    img_path = os.path.join(class_path, img_name)
                    print(f"{'='*70}")
                    print(f"Görüntü: {img_path}")
                    predicted, confidence, probs = predict_single_image(
                        img_path, model, class_names, device
                    )
                    print(f"Gerçek Sınıf: {class_name}")
                    print(f"Tahmin: {predicted} (Güven: {confidence*100:.2f}%)")
                    print(f"{'='*70}\n")
else:
    print("⚠️ Test klasörü bulunamadı.")


In [ ]:
print("\n" + "="*70)
print("PROJE ÖZETİ")
print("="*70)
print(f"Model: CNN (MedicalImageCNN)")
print(f"Sınıf Sayısı: {num_classes}")
print(f"Sınıflar: {class_names}")
print(f"\nEğitim Sonuçları (Validation):")
print(f"  Accuracy: {checkpoint.get('val_acc', 0):.4f}")
print(f"  AUC-ROC: {checkpoint.get('val_auc_roc', 0):.4f}")
print(f"  Epoch: {checkpoint.get('epoch', 'N/A')}")
print(f"\nModel Dosyası: {CHECKPOINT_PATH}")
print(f"Grafikler:")
print(f"  - checkpoints/training_history.png")
print(f"  - checkpoints/confusion_matrix.png")
print("="*70)


In [ ]:
image_path = 'test/TURBERCULOSIS/Tuberculosis-660.png'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])

image = Image.open(image_path).convert('RGB')
image_tensor = transform(image).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    outputs = model(image_tensor)
    probabilities = F.softmax(outputs, dim=1)
    confidence, predicted = torch.max(probabilities, 1)

predicted_class = class_names[predicted.item()]
confidence_score = confidence.item()

all_probs = probabilities[0].cpu().numpy()
prob_dict = {class_names[i]: float(all_probs[i]) for i in range(len(class_names))}

print("="*50)
print("Tahmin Sonuçları:")
print("="*50)
print(f"Görüntü: {image_path}")
print(f"Tahmin Edilen Sınıf: {predicted_class}")
print(f"Güven Seviyesi: {confidence_score:.4f} ({confidence_score*100:.2f}%)")
print("\nTüm sınıflar için olasılıklar:")
sorted_probs = sorted(prob_dict.items(), key=lambda x: x[1], reverse=True)
for class_name, prob in sorted_probs:
    print(f"  {class_name}: {prob:.4f} ({prob*100:.2f}%)")
print("="*50)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title(f'Görüntü: {os.path.basename(image_path)}')
plt.axis('off')

plt.subplot(1, 2, 2)
classes = [item[0] for item in sorted_probs]
probs = [item[1] for item in sorted_probs]
plt.barh(classes, probs, color='steelblue')
plt.xlabel('Olasılık')
plt.title(f'Tahmin: {predicted_class} ({confidence_score*100:.2f}%)')
plt.xlim([0, 1])
for i, prob in enumerate(probs):
    plt.text(prob + 0.01, i, f'{prob:.4f}', va='center')
plt.tight_layout()
plt.show()


![alt text](checkpoints/confusion_matrix.png)

![alt text](checkpoints/training_history.png)

![alt text](<Ekran görüntüsü 2025-12-30 140832.png>)